## Uses interpolated grid of lmn to add estimated dips to all contacts

<font face color='red'>Best with basal contacts only, using Notebook #1 as this avoids intrusive  and fault contacts being considered</font>

In [4]:
import os
#print(os.getcwd())
os.chdir('../map2loop')

print(os.getcwd())
from map2loop import m2l_interpolation
import pandas as pd
import rasterio
import numpy as np

C:\Users\00073294\Dropbox\1_Jupyter_notebooks\map2loop\map2loop


In [5]:
mname='hams3' #root name of model input data
minx=500057    #left hams
maxx=603028    #right
miny=7455348   #bottom
maxy=7567953   #top
src_crs = 'epsg:4326'  #input data coord ref system (assumed to be geodetic lat/long WGS84)
dst_crs = 'epsg:28350' #model ref system (assumed to be something with metre coordinates)
gcode='GROUP_'
dcode='DIP'
ddcode='DIP_DIR'
ccode='CODE'
scheme='scipy_rbf'
gridx=gridy=25
inset=0
this_gcode=('Hamersley Group','Fortescue Group','Turee Group')


test_data_path='../test_data3/'

tmp_path=test_data_path+'tmp/'
data_path=test_data_path+'data/'
dtm_path=test_data_path+'dtm/'
output_path=test_data_path+'output/'
dtm_file = dtm_path+'hams3_dtm_rp.tif'


geology_file=tmp_path+'basal_contacts2.shp'
fault_file=tmp_path+'faults_clip.shp'

bbox=(minx+inset,miny+inset,maxx-inset,maxy-inset)

In [7]:
dtm=dataset = rasterio.open(dtm_path+'/'+mname+'_dtm_rp.tif')

In [8]:
lo=np.loadtxt(tmp_path+'interpolation_l.csv',skiprows =1,delimiter =',',dtype=float)
mo=np.loadtxt(tmp_path+'interpolation_m.csv',skiprows =1,delimiter =',',dtype=float)
no=np.loadtxt(tmp_path+'interpolation_n.csv',skiprows =1,delimiter =',',dtype=float)
xy=np.loadtxt(tmp_path+'interpolation_'+scheme+'.csv',skiprows =1,delimiter =',',dtype=float)

NameError: name 'np' is not defined

In [ ]:
def add_dips_to_contacts(geology_file,output_path,bbox,dcode,ddcode,gcode,calc):
    print(geology_file,output_path,bbox,dcode,ddcode,gcode,calc,gridx,gridy)
    geol_file = gpd.read_file(geology_file,bbox=bbox)
    print(len(geol_file))
    geol_file.plot( color='black',edgecolor='black') 
    
    # Setup: Generate data...
    npts = 0
    decimate=1
    nx, ny = gridx,gridy

    f=open(output_path+'contacts_with_dips.csv','w')
    f.write("x,y,z,polarity,formation\n")
    j=0
    i=0
    for acontact in geol_file.iterrows():   #loop through distinct linestrings in MultiLineString
        if(acontact[1].geometry.type=='MultiLineString'):
            #print(i)
            for line in acontact[1].geometry: # loop through line segments
                #print(i,len(acontact[1].geometry))
                if(i%decimate ==0):
                    #if(acontact[1]['id']==170): 
                        #display(npts,line.coords[0][0],line.coords[1][0]) 
                    dlsx=line.coords[0][0]-line.coords[1][0]
                    dlsy=line.coords[0][1]-line.coords[1][1]
                    if(not line.coords[0][0]==line.coords[1][0] or not line.coords[0][1]==line.coords[1][1]):               
                        lsx=dlsx/sqrt((dlsx*dlsx)+(dlsy*dlsy))
                        lsy=dlsy/sqrt((dlsx*dlsx)+(dlsy*dlsy))
                        x[i]=line.coords[1][0]+(dlsx/2)
                        y[i]=line.coords[1][1]+(dlsy/2)
                        angle=degrees(atan2(lsx,lsy))
                        l[i]=lsx
                        m[i]=lsy
                        ostr=str(x[i])+","+str(y[i])+","+str(angle%180)+","+str(lsx)+","+str(lsy)+"\n"
                        #print(ostr)
                        f.write(ostr)
                        npts=npts+1
                i=i+1
        else:
            #display(acontact[1].geometry,acontact[1].geometry.coords)
            #for line in acontact[1]: # loop through line segments in LineString
            if(  i%decimate ==0):
                dlsx=acontact[1].geometry.coords[0][0]-acontact[1].geometry.coords[1][0]
                dlsy=acontact[1].geometry.coords[0][1]-acontact[1].geometry.coords[1][1]
                if(not acontact[1].geometry.coords[0][0]==acontact[1].geometry.coords[1][0] 
                   or not acontact[1].geometry.coords[0][1]==acontact[1].geometry.coords[1][1]):
                    lsx=dlsx/sqrt((dlsx*dlsx)+(dlsy*dlsy))
                    lsy=dlsy/sqrt((dlsx*dlsx)+(dlsy*dlsy))
                    x[i]=acontact[1].geometry.coords[1][0]+(dlsx/2)
                    y[i]=acontact[1].geometry.coords[1][1]+(dlsy/2)
                    angle=degrees(atan2(lsx,lsy))
                    l[i]=lsx
                    m[i]=lsy
                    ostr=str(x[i])+","+str(y[i])+","+str(angle%180)+","+str(lsx)+","+str(lsy)+"\n"
                    #print(ostr)
                    f.write(ostr)
                    print(npts,dlsx,dlsy)
                    npts=npts+1
                i=i+1
        j=j+1
    f.close()
    print("i",i,"npts",npts)

    for i in range(0,npts):
        x[i]=x[i]+(np.random.ranf()*0.01)
        y[i]=y[i]+(np.random.ranf()*0.01)

    ZIl,ZIm,ZIn=call_interpolator(calc,x[:npts],y[:npts],l[:npts],m[:npts],0,xi,yi,nx,ny)    
    
    # Comparisons...
    plot(x,y,l,ZIl)
    plt.title('l')
    plot(x,y,m,ZIm)
    plt.title('m')

    fi=open(output_path+'interpolation_contacts_'+calc+'.csv','w')
    fl=open(output_path+'interpolation_contacts_l.csv','w')
    fm=open(output_path+'interpolation_contacts_m.csv','w')
    
    fi.write("x,y,angle\n")
    fl.write("x,y,l\n")
    fm.write("x,y,m\n")
    
    for xx in range (0,gridx-1):
        for yy in range (0,gridy-1):
            yyy=xx
            xxx=gridy-2-yy
            L=ZIl[xxx,yyy]/(sqrt((pow(ZIl[xxx,yyy],2.0))+(pow(ZIm[xxx,yyy],2.0))))
            M=ZIm[xxx,yyy]/(sqrt((pow(ZIl[xxx,yyy],2.0))+(pow(ZIm[xxx,yyy],2.0))))
            S=degrees(atan2(L,M))
    
            ostr=str(bbox[0]+(xx*((bbox[2]-bbox[0])/100)))+","+str(bbox[1]+((gridy-2-yy)*((bbox[3]-bbox[1])/100)))+","+str(int(S))+'\n'
            fi.write(ostr)
            
            ostr=str(xx)+","+str(yy)+","+str(L)+'\n'
            fl.write(ostr)
            ostr=str(xx)+","+str(yy)+","+str(M)+'\n'
            fm.write(ostr)
    fi.close()
    fl.close()
    fm.close()
    fig, ax = plt.subplots(figsize=(10, 10))
    q = ax.quiver(xi, yi, ZIl, ZIm,headwidth=0)
    plt.show()